# A1111 Extension Installer

This notebook installs **Automatic1111** extensions into `/workspace/a1111/extensions` (configurable).

**How it works**
- **Defaults**: always installed (safe, lightweight helpers)
- **Recommended**: toggle `enable: true/false` per extension
- **Extra URLs**: paste any Git repo URLs to install as well

**Notes**
- Running the notebook again is safe: it will `git pull` existing extensions.
- After changes, **Reload UI** in WebUI (Settings → Restart UI) to load new extensions.


In [ ]:
# --- Paths / settings ---
from pathlib import Path
A1111_ROOT = Path("/workspace/a1111")      # change if you moved WEBUI_ROOT
EXT_DIR = A1111_ROOT / "extensions"

EXT_DIR.mkdir(parents=True, exist_ok=True)
print("Extensions directory:", EXT_DIR)

# --- Defaults: always install ---
DEFAULT_EXTS = [
    # Lightweight, broadly useful
    {"name": "Tag Autocomplete", "url": "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"},
    {"name": "Images Browser", "url": "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"}
]

# --- Recommended: toggle enable True/False ---
RECOMMENDED = [
    {"name": "ControlNet", "url": "https://github.com/Mikubill/sd-webui-controlnet", "enable": false},
    {"name": "ADetailer", "url": "https://github.com/Bing-su/adetailer", "enable": true},
    {"name": "Ultimate SD Upscale", "url": "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111", "enable": true},
    {"name": "Dynamic Prompts", "url": "https://github.com/adieyal/sd-dynamic-prompts", "enable": false},
    {"name": "Openpose Editor", "url": "https://github.com/fkunn1326/openpose-editor", "enable": false}
]

# --- Extra URLs: paste any additional Git repos here ---
EXTRA_URLS = [
    # e.g. "https://github.com/someuser/some-extension"
]
print("Configured:", len(DEFAULT_EXTS), "defaults,",
      sum(1 for r in RECOMMENDED if r.get("enable")), "recommended enabled,",
      len(EXTRA_URLS), "extra URLs")


In [ ]:
import subprocess, sys, re
from pathlib import Path

def run(cmd, cwd=None):
    print("$", " ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=True)

def repo_dir_for(url: str, base: Path) -> Path:
    # derive a folder name from the repo URL (owner/repo)
    m = re.search(r"github\.com[:/]{1,2}([^/]+)/([^/.]+)", url)
    if m:
        return base / f"{m.group(1)}__{m.group(2)}"
    # fallback: sanitize full URL
    safe = re.sub(r"[^a-zA-Z0-9_.-]", "_", url)
    return base / safe

def install_or_update(url: str, base: Path):
    dst = repo_dir_for(url, base)
    if not dst.exists():
        print(f"\n[install] {url} → {dst}")
        run(["git", "clone", "--depth", "1", url, str(dst)])
    else:
        print(f"\n[update] {dst}")
        # clean + pull fast-forward if possible
        try:
            run(["git", "-C", str(dst), "reset", "--hard"]) 
            run(["git", "-C", str(dst), "pull", "--ff-only"]) 
        except subprocess.CalledProcessError:
            print("  (non-ff merge; attempting rebase)")
            run(["git", "-C", str(dst), "pull", "--rebase", "--autostash"]) 
    return dst

def process_all():
    installed = []
    errors = []
    # 1) defaults
    for ext in DEFAULT_EXTS:
        try:
            installed.append(install_or_update(ext["url"], EXT_DIR))
        except Exception as e:
            errors.append((ext["url"], str(e)))
    # 2) recommended (enabled only)
    for rec in RECOMMENDED:
        if rec.get("enable"):
            try:
                installed.append(install_or_update(rec["url"], EXT_DIR))
            except Exception as e:
                errors.append((rec["url"], str(e)))
    # 3) extras
    for url in EXTRA_URLS:
        try:
            installed.append(install_or_update(url, EXT_DIR))
        except Exception as e:
            errors.append((url, str(e)))
    print("\n=== Summary ===")
    for p in installed:
        print("✔", p)
    if errors:
        print("\nErrors:")
        for url, msg in errors:
            print("✖", url, "→", msg)

process_all()


## Next steps
- In the WebUI, open **Settings → Reload UI** to load installed extensions.
- If an extension requires models (e.g. ControlNet), use a models notebook or download to:
  - `/workspace/a1111/models/ControlNet`
- Rerun this notebook any time to update extensions.